<h3> General Mapping Idea: </h3>
  
**Combined Rule-based Matching Strategy:** two sources of information: telephone number,  geological information (address & geo) <br>
1). parse geological information to retrive country, state, zip, etc., data of entities <br>
2). normalize telephone numbers (and ideally decompose telephone numbers into country code, area code, and national number for better matching outcomes)

### 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from os import listdir
from os.path import isfile, join
import json
import gzip
import shutil 

# import geopy.distance

 ### 2. Define Functions

In [2]:
# concatenate dataframes
def concatenate_dataframe(source_path):
    files = os.listdir(source_path)
    # initialize a dataframe
    df_final = pd.DataFrame()
    for file in files:
        df = pd.read_json(os.path.join(source_path, file), compression='gzip', lines=True)
        df['table_id'] = file.strip('.json.gz')
        df_final = pd.concat([df_final, df], ignore_index=True)
        
    return df_final

<b> Geological Information

In [3]:
# preprocessing
class EntityPreprocessor:
    """
    iterator over cells in a column to flatten all dictionary-typed objects, 
    then extract values of all keys and save the values in the newly-added corresponding column
    
    Args:
    df: dataframe to be preprocessed 
    cols: columns to be flattened
    keys: a dictionary to store keys that have appeared in the columns
    
    Returns:
    a new dataframe with appended columns
    a dictionary store keys in user-specified columns
    """
    
    def __init__(self, df, cols=[], keys={}):
        self.df = df
        self.cols = cols
        self.keys = keys
    
    # collect keys 
    def collect_keys(self):
        df = self.df
        cols = self.cols
        keys = self.keys
        
        for col in cols:
            if not col in df.columns:
                continue
            # drop records without required data
            tmp = df[~df[col].isna()][col]
            # initialize a list to store keys appearing in the current column
            result_list = []
            for item in tmp:
                try:
                    for key in item.keys():
                        if not key in result_list:
                            result_list.append(key)
                except:
                    continue
            
            # append to the dictionary
            keys[col] = result_list
            
        self.keys = keys
        return keys

    # flatten a dictionary
    @staticmethod
    def _extract_value(iterator, key):
        if isinstance(iterator, dict): 
            return iterator.get(key, None)
        else: 
            return None

    # append new columns to store extracted infomation
    def column_parser(self, inplace=False): 
        df = self.df
        cols = self.cols
        keys = self.keys
        if keys == {}:
            raise Error("self.keys is an empty list. Please collect keys first")
        
        for col in cols:
            if not col in df.columns:
                continue
            keys_list = keys[col]
            for key in keys_list:
                # column_parser.key = key
                if key != 'telephone':
                    df[key] = None
                    df[key] = df[col].apply(EntityPreprocessor._extract_value, args=(key,))
                else:
                    df['telephone_'+ col] = df[col].apply(EntityPreprocessor._extract_value, args=(key,))
        if inplace:
            self.df = df
        else: 
            return df

In [4]:
# geo distance calculation
def geo_distance(coords_1, coords_2):
    return geopy.distance.vincenty(coords_1, coords_2).km

<b> Telephone Number

In [54]:
# telephone preprocessing


<b> Entity Name 

In [5]:
# jaccard similarity calculation
def jaccard_distance(s1, s2):
    tokenized_s1 = set(s1.split(' '))
    tokenized_s2 = set(s2.split(' '))
    overlap = tokenized_s1.intersection(tokenized_s2) 
    union = tokenized_s1.union(tokenized_s2) 
    
    return len(overlap)/len(union)    

### 3. Exploratory Analysis using Top100 Tables

In [4]:
path_parent = os.path.dirname(os.getcwd())
data_path = os.path.join(path_parent, 'src/data')

In [21]:
path_parent

'/work-ceph/bizer-tp2021/data_integration_using_deep_learning'

In [6]:
cd ../src/data/LocalBusiness/LocalBusiness_rest

/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/LocalBusiness/LocalBusiness_rest


In [7]:
cd geo_preprocessed

/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/LocalBusiness/LocalBusiness_rest/geo_preprocessed


In [8]:
ls

LocalBusiness_000game.com_September2020.json.gz
LocalBusiness_00100100.com_September2020.json.gz
LocalBusiness_007airportlimodenver.com_September2020.json.gz
LocalBusiness_007bailbondsinvestigations.com_September2020.json.gz
LocalBusiness_007electricians.com_September2020.json.gz
LocalBusiness_01-4.com_September2020.json.gz
LocalBusiness_01-5.com_September2020.json.gz
LocalBusiness_01crypto.com_September2020.json.gz
LocalBusiness_01generator.com_September2020.json.gz
LocalBusiness_0-1s.com_September2020.json.gz
LocalBusiness_024hunyan.com_September2020.json.gz
LocalBusiness_02-5.com_September2020.json.gz
LocalBusiness_0260cars.com_September2020.json.gz
LocalBusiness_02-9.com_September2020.json.gz
LocalBusiness_02patroldivisionsecurity.com_September2020.json.gz
LocalBusiness_03-3.com_September2020.json.gz
LocalBusiness_0-34.com_September2020.json.gz
LocalBusiness_03-5.com_September2020.json.gz
LocalBusiness_04-2.com_September2020.json.gz
LocalBusiness_04-3.com_September2020.json.gz
Loca

LocalBusiness_24one.com_September2020.json.gz
LocalBusiness_24online.net_September2020.json.gz
LocalBusiness_24options.co.uk_September2020.json.gz
LocalBusiness_24rooter.com_September2020.json.gz
LocalBusiness_24roulette.com_September2020.json.gz
LocalBusiness_24sevenhomehealth.com_September2020.json.gz
LocalBusiness_24sevensameday.co.uk_September2020.json.gz
LocalBusiness_24thandcollins.com_September2020.json.gz
LocalBusiness_24v.org_September2020.json.gz
LocalBusiness_24webcam.com_September2020.json.gz
LocalBusiness_24webtech.com_September2020.json.gz
LocalBusiness_24x7air.com_September2020.json.gz
LocalBusiness_24x7page.com_September2020.json.gz
LocalBusiness_24x7techservices.com_September2020.json.gz
LocalBusiness_2501digital.com_September2020.json.gz
LocalBusiness_253oldwharf.com_September2020.json.gz
LocalBusiness_256w36.com_September2020.json.gz
LocalBusiness_25auto.com_September2020.json.gz
LocalBusiness_25broadmoorapts.com_September2020.json.gz
LocalBusiness_25db.com_September

LocalBusiness_acscleaningltd.co.uk_September2020.json.gz
LocalBusiness_acscleaningmelbourne.com.au_September2020.json.gz
LocalBusiness_acscollision.com_September2020.json.gz
LocalBusiness_acscontractingservices.com_September2020.json.gz
LocalBusiness_acscorporation.com_September2020.json.gz
LocalBusiness_acscreative.com_September2020.json.gz
LocalBusiness_acscva.com_September2020.json.gz
LocalBusiness_acsda.com_September2020.json.gz
LocalBusiness_acsdoorservices.net.au_September2020.json.gz
LocalBusiness_acs-dxb.com_September2020.json.gz
LocalBusiness_acsentnow.com_September2020.json.gz
LocalBusiness_acservicecenter.net_September2020.json.gz
LocalBusiness_acservicelakecharles.com_September2020.json.gz
LocalBusiness_acsexec.com_September2020.json.gz
LocalBusiness_acsexpress.com_September2020.json.gz
LocalBusiness_acsflighttraining.co.uk_September2020.json.gz
LocalBusiness_acsfoothills.com_September2020.json.gz
LocalBusiness_acsgainesville.com_September2020.json.gz
LocalBusiness_acsgulfc

LocalBusiness_allamericanhomeinspectionllc.com_September2020.json.gz
LocalBusiness_allamericanjiujitsu.com_September2020.json.gz
LocalBusiness_allamericank9llc.com_September2020.json.gz
LocalBusiness_allamericanlandscape.com_September2020.json.gz
LocalBusiness_allamericanperformance.com_September2020.json.gz
LocalBusiness_allamericanpestcontrol.com_September2020.json.gz
LocalBusiness_allamericanpetresorts.com_September2020.json.gz
LocalBusiness_allamericanpropertyinspection.com_September2020.json.gz
LocalBusiness_allamericanrfg.com_September2020.json.gz
LocalBusiness_allamericantechnicianacademy.com_September2020.json.gz
LocalBusiness_allamericantowingtx.com_September2020.json.gz
LocalBusiness_allamericantractorandtree.com_September2020.json.gz
LocalBusiness_allamericantrailer.com_September2020.json.gz
LocalBusiness_allamericantreeplus.com_September2020.json.gz
LocalBusiness_allamericantreeserviceflorida.com_September2020.json.gz
LocalBusiness_allamericanwastenm.com_September2020.json.

LocalBusiness_artechcondosforsale.com_September2020.json.gz
LocalBusiness_artechrealtors.com_September2020.json.gz
LocalBusiness_artechroof.com_September2020.json.gz
LocalBusiness_artecitycondosforsale.com_September2020.json.gz
LocalBusiness_arteclighting.com_September2020.json.gz
LocalBusiness_arteconalas.com_September2020.json.gz
LocalBusiness_arte-decoracion-floral.com_September2020.json.gz
LocalBusiness_artedelgusto.com_September2020.json.gz
LocalBusiness_artedentalessex.com_September2020.json.gz
LocalBusiness_artedentalkids.com_September2020.json.gz
LocalBusiness_artederadi.com_September2020.json.gz
LocalBusiness_artedeser.com_September2020.json.gz
LocalBusiness_arteecorpinturaemlaca.com_September2020.json.gz
LocalBusiness_arteengalerias.com_September2020.json.gz
LocalBusiness_arteestile.com_September2020.json.gz
LocalBusiness_artefactindustries.com_September2020.json.gz
LocalBusiness_artefactosdecobre.com.mx_September2020.json.gz
LocalBusiness_arteffect.co.uk_September2020.json.g

...

LocalBusiness_bjcohen.com_September2020.json.gz
LocalBusiness_bjcomfortsolutions.com_September2020.json.gz
LocalBusiness_b-j.com_September2020.json.gz
LocalBusiness_bj-digital.com_September2020.json.gz
LocalBusiness_bjduffyfuneralhome.com_September2020.json.gz
LocalBusiness_bjertnes.com_September2020.json.gz
LocalBusiness_bje.uk_September2020.json.gz
LocalBusiness_bjgoulding.co.uk_September2020.json.gz
LocalBusiness_bjhsg.org_September2020.json.gz
LocalBusiness_bjhxxyjj.com.cn_September2020.json.gz
LocalBusiness_bjiebitcoin.com_September2020.json.gz
LocalBusiness_bjkpainting.com.au_September2020.json.gz
LocalBusiness_bjlfinancialservices.com_September2020.json.gz
LocalBusiness_bjmcnamara.com.au_September2020.json.gz
LocalBusiness_bjmco.com_September2020.json.gz
LocalBusiness_bjn.com_September2020.json.gz
LocalBusiness_bjorklundlaw.com_September2020.json.gz
LocalBusiness_bjpeachconstructions.com.au_September2020.json.gz
LocalBusiness_bjqzlyj.com_September2020.json.gz
LocalBusiness_bjrad

LocalBusiness_bpsroadside.com_September2020.json.gz
LocalBusiness_bpssoutheast.com_September2020.json.gz
LocalBusiness_bpstore.com_September2020.json.gz
LocalBusiness_bpsunshineroofers.com_September2020.json.gz
LocalBusiness_bpsystems.co.uk_September2020.json.gz
LocalBusiness_b-pt.com_September2020.json.gz
LocalBusiness_bpti.com_September2020.json.gz
LocalBusiness_bptowing.com_September2020.json.gz
LocalBusiness_bptrade.com_September2020.json.gz
LocalBusiness_bpublic.com_September2020.json.gz
LocalBusiness_bpvending.com_September2020.json.gz
LocalBusiness_bpvsolar.co.uk_September2020.json.gz
LocalBusiness_bpxo.com_September2020.json.gz
LocalBusiness_bpyp.com_September2020.json.gz
LocalBusiness_bqe.com.au_September2020.json.gz
LocalBusiness_bqe.com_September2020.json.gz
LocalBusiness_bqfurniturestore.com_September2020.json.gz
LocalBusiness_bqmaids.com_September2020.json.gz
LocalBusiness_bqor.com_September2020.json.gz
LocalBusiness_bquant.com_September2020.json.gz
LocalBusiness_br8kout.c

LocalBusiness_bushflooringcenter.com_September2020.json.gz
LocalBusiness_bushfuneralhomes.com_September2020.json.gz
LocalBusiness_bushfurnitureman.com_September2020.json.gz
LocalBusiness_bushidokai.co.uk_September2020.json.gz
LocalBusiness_bushidokan.com_September2020.json.gz
LocalBusiness_bushmantanks.com.au_September2020.json.gz
LocalBusiness_bushmillsinn.com_September2020.json.gz
LocalBusiness_bushnellandbushnell.com_September2020.json.gz
LocalBusiness_bushnellanimalclinic.com_September2020.json.gz
LocalBusiness_bushskinz4x4.com.au_September2020.json.gz
LocalBusiness_bushstudio.com_September2020.json.gz
LocalBusiness_bushtaxi.com_September2020.json.gz
LocalBusiness_bushtobeachplumbingservices.com.au_September2020.json.gz
LocalBusiness_bushtribe.com_September2020.json.gz
LocalBusiness_bushwickjewelrycasting.com_September2020.json.gz
LocalBusiness_bushybusiness.com_September2020.json.gz
LocalBusiness_businenissinsidenir.com_September2020.json.gz
LocalBusiness_businertia.com_September2

LocalBusiness_careeracademy.com_September2020.json.gz
LocalBusiness_careeracademyoftraining.org_September2020.json.gz
LocalBusiness_career-advice.com_September2020.json.gz
LocalBusiness_careercenter.uk_September2020.json.gz
LocalBusiness_career-coach.com_September2020.json.gz
LocalBusiness_careercoaches.com_September2020.json.gz
LocalBusiness_careercoachingandtraining.com.au_September2020.json.gz
LocalBusiness_careercoachrichmond.com_September2020.json.gz
LocalBusiness_careercompass.co.uk_September2020.json.gz
LocalBusiness_careerconfidencecoachforwomen.co.uk_September2020.json.gz
LocalBusiness_careercounselingglenellynil.com_September2020.json.gz
LocalBusiness_careercraft.co.uk_September2020.json.gz
LocalBusiness_careerdesign.com_September2020.json.gz
LocalBusiness_careerdevelopers.com_September2020.json.gz
LocalBusiness_careerdoors.com_September2020.json.gz
LocalBusiness_careerexcel.com_September2020.json.gz
LocalBusiness_careerfax.com_September2020.json.gz
LocalBusiness_careerful.co

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# preprocessing
source_path = os.path.join(data_path, 'LocalBusiness/LocalBusiness_rest/cleaned')
target_path = os.path.join(data_path, 'LocalBusiness/LocalBusiness_rest/geo_preprocessed')
gzfiles = os.listdir(source_path) 

# for file in gzfiles: 
#     if not file.endswith('.json.gz'):
#         continue
#     df_in = pd.read_json(os.path.join(source_path, file), compression='gzip', lines=True)
#     local_business = EntityPreprocessor(df_in, cols=['address','geo'])
#     keys = local_business.collect_keys()
#     df_out = local_business.column_parser()
#     df_out.to_json(os.path.join(target_path, file), compression='gzip', orient='records', lines=True)

In [10]:
df_in = pd.read_json(os.path.join(source_path, 'LocalBusiness_000game.com_September2020.json.gz'), compression='gzip', lines=True)
df_in

,row_id,name,address,telephone,aggregaterating,page_url
0,0,DAN.COM,"{'postalcode': '1054 ET', 'streetaddress': 'Te...",+1 737 218 6046,"{'ratingvalue': '4.7', 'bestrating': '5', 'wor...",https://000game.com/


<b> 1) Local Business

In [11]:
df_out.to_json(os.path.join(target_path, 'LocalBusiness_000game.com_September2020.json.gz'), compression='gzip', orient='records', lines=True)
df_out

NameError: name 'df_out' is not defined

In [8]:
source_path = os.path.join(data_path, 'LocalBusiness/LocalBusiness_rest/cleaned')
df_lb=concatenate_dataframe(source_path).reset_index().drop(columns='index',axis=0)

local_business = EntityPreprocessor(df_lb, cols=['address','geo'])
keys = local_business.collect_keys()
df_lb_new = local_business.column_parser()

**Geo Info Preprocessing**  <br>
**Results** <br>
**1) Address Keys:**<br> 
'addressregion' (57.7%), 'postalcode'(42.8%), 'streetaddress'(52.7%),'addresslocality' (66.6%), 'addresscountry' (40.9%), 'postofficeboxnumber'(1.3%), 'citystatezip('1.3%'), 'telephone_address'(5.9%), 'faxnumber'(2.9%), 'sameas(3.1%)' 

It seems that *'addressregion', 'addresslocality', 'addresscountry', 'postalcode'* can be potentially used as identifiers. There is a python library called ***pgeocode*** that can extract more infomation from postal codes (conditional on country is known). (Ref: https://www.journaldev.com/49094/find-address-from-zip-code-in-python) <br> 

Besides, it is noticable that most records in 'addressregion' refer to states of different countries, which, however, might be named with synonyms. The normalized *'addressregion'* values can be used to select entties from a given country. Then the updated column *'addresscountry'* can be further be used during the parsing of telephone numbers and postal codes. 

**2) Geo Keys:** <br>
'longitude'('25.1%'), 'latitude'(30.4%)

The longitudes and latitudes of entities might refer to those of entity headquarters. Notice that *'address* often refers to a specific entity branch, a better decision is probably to replace the longitudes and latitudes from those extracted from postal codes using ***pgepcode***